In [26]:
import pandas as pd
bus_text_file = "/Users/joeyrich/Desktop/local/varseek/output_bus_sample.txt"
chunksize = sys.maxsize

for i, bus_df in enumerate(pd.read_csv(bus_text_file, sep="\t", header=None, names=["barcode", "UMI", "EC", "count", "read_index"], chunksize=chunksize)):
    print(bus_df)
    print("------------------")
    print("------------------")
    print("------------------")

                                             barcode  UMI  EC  count  \
0  AAAAAAAAAAAAAAAA        T       0       1     ...  NaN NaN    NaN   
1  AAAAAAAAAAAAAAAA        T       1       1     ...  NaN NaN    NaN   
2  AAAAAAAAAAAAAAAA        T       2       1     ...  NaN NaN    NaN   
3  AAAAAAAAAAAAAAAA        T       2       1     ...  NaN NaN    NaN   
4  AAAAAAAAAAAAAAAA        T       3       1     ...  NaN NaN    NaN   
5  AAAAAAAAAAAAAAAA        T       4       1     ...  NaN NaN    NaN   
6  AAAAAAAAAAAAAAAA        T       4       1     ...  NaN NaN    NaN   
7  AAAAAAAAAAAAAAAA        T       4       1     ...  NaN NaN    NaN   

   read_index  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
5         NaN  
6         NaN  
7         NaN  
------------------
------------------
------------------


In [16]:
bus_df

,barcode,UMI,EC,read_index
0,AAAAAAAAAAAAAAAA,T,0,588
1,AAAAAAAAAAAAAAAA,T,1,656
2,AAAAAAAAAAAAAAAA,T,2,886
3,AAAAAAAAAAAAAAAA,T,2,997
4,AAAAAAAAAAAAAAAA,T,3,4353
5,AAAAAAAAAAAAAAAA,T,4,4440
6,AAAAAAAAAAAAAAAA,T,4,4471
7,AAAAAAAAAAAAAAAA,T,4,4490


In [3]:
import importlib
import varseek.utils.varseek_clean_utils
importlib.reload(varseek.utils.varseek_clean_utils)
from varseek.utils.varseek_clean_utils import make_bus_df

In [4]:
kb_count_out_dir = '/Users/joeyrich/Desktop/local/RLSRWP_2025/data/varseek_count_out_alignment_visualization_full/kb_count_out_vcrs'
fastq_file = '/Users/joeyrich/Desktop/local/RLSRWP_2025/data/ccle_data_base/RNASeq_MELHO_SKIN/SRR8615233_1.fastq.gz'
technology = 'bulk'
parity = 'single'
bustools = '/Users/joeyrich/miniconda3/envs/RLSRWP_2025/lib/python3.10/site-packages/kb_python/bins/darwin/m1/bustools/bustools'

In [5]:
bus_df = make_bus_df(kb_count_out_dir, fastq_file, technology=technology, parity=parity, bustools=bustools)
# bus_df = bus_df.drop(columns=["transcript_names"]).rename(columns={"gene_names": "variant_names"})
# filtered_bus_df = bus_df[bus_df["variant_names"].apply(lambda x: len(x) == 1)]  # remove multi-mapping reads
# filtered_bus_df["variant_names_str"] = filtered_bus_df["variant_names"].apply(lambda x: x[0])  # cast to string
# filtered_bus_df = filtered_bus_df[(filtered_bus_df["variant_names_str"].isin(top10)) & (filtered_bus_df["counted_in_count_matrix"])]
# print(f"Number of reads in filtered bus file: {len(filtered_bus_df)}")
# filtered_bus_df.head()

loading in transcripts
loading in barcodes


Processing FASTQ headers: 69015721it [04:57, 231875.48it/s]


loading in ec matrix
loading in bus df


AttributeError: 'str' object has no attribute 'drop'

In [1]:
import os
import subprocess
import shutil
import sys
import pandas as pd
from varseek.utils import create_identity_t2g

varseek_directory = os.path.dirname(os.path.abspath(""))

conda_env_path = os.path.dirname(os.path.dirname(shutil.which("kb")))  # to get kb path
operating_system = "linux" if sys.platform.startswith("linux") else "darwin/m1"

kallisto = f"{conda_env_path}/lib/python3.10/site-packages/kb_python/bins/{operating_system}/kallisto/kallisto"  # or kallisto_k64
bustools = f"{conda_env_path}/lib/python3.10/site-packages/kb_python/bins/{operating_system}/bustools/bustools"

In [2]:
ref_fa = f"{varseek_directory}/tests/kb_files/ref_test.fa"
read1_fq = f"{varseek_directory}/tests/kb_files/reads1_test.fq"
read2_fq = f"{varseek_directory}/tests/kb_files/reads2_test.fq"
test_index = f"{varseek_directory}/tests/kb_files/index_test.idx"
test_t2g = f"{varseek_directory}/tests/kb_files/t2g_test.txt"
kb_count_out_test = f"{varseek_directory}/tests/kb_files/test_kb_count_out_March2025_with_num"
kb_count_out_test2 = f"{varseek_directory}/tests/kb_files/test_kb_count_out_March2025_without_num"

rnaseq_fastq_files_final = [read1_fq, read2_fq]
os.makedirs(kb_count_out_test, exist_ok=True)

if not os.path.exists(test_index):
    kb_ref_command = ["kb", "ref", "--workflow", "custom", "-t", str(8), "-i", test_index, "--d-list", "None", ref_fa]
    subprocess.run(kb_ref_command, check=True)

if not os.path.exists(test_t2g):
    create_identity_t2g(ref_fa, test_t2g)

if not os.path.exists(kb_count_out_test) or len(os.listdir(kb_count_out_test)) == 0:
    kb_count_command = ["kb", "count", "-t", str(2), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "single", "-o", kb_count_out_test, "--kallisto", kallisto, "--bustools", bustools] + rnaseq_fastq_files_final
    subprocess.run(kb_count_command, check=True)

if not os.path.exists(kb_count_out_test2) or len(os.listdir(kb_count_out_test2)) == 0:
    kb_count_command = ["kb", "count", "-t", str(2), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--h5ad", "--parity", "single", "-o", kb_count_out_test2, "--kallisto", kallisto, "--bustools", bustools] + rnaseq_fastq_files_final
    subprocess.run(kb_count_command, check=True)

# kb_count_out_test = f"{varseek_directory}/tests/kb_files/test_kb_count_out_paired"
# if not os.path.exists(kb_count_out_test) or len(os.listdir(kb_count_out_test)) == 0:
#     kb_count_command = ["kb", "count", "-t", str(8), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "paired", "-o", kb_count_out_test] + rnaseq_fastq_files_final
#     subprocess.run(kb_count_command, check=True)

In [ ]:
bus_text_file = os.path.join(kb_count_out_test, "bus_text.txt")
bus_file = os.path.join(kb_count_out_test, "output.bus")
bustools_text_command = f"{bustools} text -o {bus_text_file} -f {bus_file}"
subprocess.run(bustools_text_command, check=True, shell=True)

bus_text_file = os.path.join(kb_count_out_test2, "bus_text.txt")
bus_file = os.path.join(kb_count_out_test2, "output.bus")
bustools_text_command = f"{bustools} text -o {bus_text_file} -f {bus_file}"
subprocess.run(bustools_text_command, check=True, shell=True)

In [ ]:
import importlib
import varseek.utils.seq_utils
importlib.reload(varseek.utils.varseek_clean_utils)
from varseek.utils.varseek_clean_utils import make_bus_df
bus_df = make_bus_df(kallisto_out = kb_count_out_test, fastq_file_list = rnaseq_fastq_files_final, t2g_file = test_t2g, mm = False, union = False, technology = "bulk", bustools = bustools)

In [5]:
read1_sample2_fq = f"{varseek_directory}/tests/kb_files/reads1_sample2_test.fq"
read2_sample2_fq = f"{varseek_directory}/tests/kb_files/reads2_sample2_test.fq"
rnaseq_fastq_files_final = [read1_fq, read2_fq, read1_sample2_fq, read2_sample2_fq]

kb_count_out_test_4_total_fastqs = f"{varseek_directory}/tests/kb_files/test_kb_count_out_March2025_with_num_4_total_fastqs"

if not os.path.exists(kb_count_out_test_4_total_fastqs) or len(os.listdir(kb_count_out_test_4_total_fastqs)) == 0:
    kb_count_command = ["kb", "count", "-t", str(2), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "single", "-o", kb_count_out_test_4_total_fastqs, "--kallisto", kallisto, "--bustools", bustools] + rnaseq_fastq_files_final
    subprocess.run(kb_count_command, check=True)

bus_text_file = os.path.join(kb_count_out_test_4_total_fastqs, "bus_text.txt")
bus_file = os.path.join(kb_count_out_test_4_total_fastqs, "output.bus")
bustools_text_command = f"{bustools} text -o {bus_text_file} -f {bus_file}"
subprocess.run(bustools_text_command, check=True, shell=True)

import importlib
import varseek.utils.seq_utils
importlib.reload(varseek.utils.varseek_clean_utils)
from varseek.utils.varseek_clean_utils import make_bus_df
bus_df = make_bus_df(kallisto_out = kb_count_out_test_4_total_fastqs, fastq_file_list = rnaseq_fastq_files_final, t2g_file = test_t2g, mm = False, union = False, technology = "bulk", bustools = bustools)

loading in transcripts
loading in barcodes
loading in fastq headers
loading in fastq headers
loading in fastq headers
loading in fastq headers
loading in ec matrix
loading in t2g df
running bustools text
loading in bus df
merging ec df into bus df
Apply the mapping function to create gene name columns
added counted in matrix column
saving bus df


Read in 12 BUS records


In [3]:
read1_sample2_fq = f"{varseek_directory}/tests/kb_files/reads1_sample2_test.fq"
read2_sample2_fq = f"{varseek_directory}/tests/kb_files/reads2_sample2_test.fq"
rnaseq_fastq_files_final = [read1_fq, read2_fq, read1_sample2_fq, read2_sample2_fq]

kb_count_out_test_4_total_fastqs_paired = f"{varseek_directory}/tests/kb_files/test_kb_count_out_March2025_with_num_4_total_fastqs_paired2"

# if not os.path.exists(kb_count_out_test_4_total_fastqs_paired) or len(os.listdir(kb_count_out_test_4_total_fastqs_paired)) == 0:
#     kb_count_command = ["kb", "count", "-t", str(2), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "paired", "-o", kb_count_out_test_4_total_fastqs_paired, "--kallisto", kallisto, "--bustools", bustools] + rnaseq_fastq_files_final
#     subprocess.run(kb_count_command, check=True)

# bus_text_file = os.path.join(kb_count_out_test_4_total_fastqs_paired, "bus_text.txt")
# bus_file = os.path.join(kb_count_out_test_4_total_fastqs_paired, "output.bus")
# bustools_text_command = f"{bustools} text -o {bus_text_file} -f {bus_file}"
# subprocess.run(bustools_text_command, check=True, shell=True)

# import importlib
# import varseek.utils.seq_utils
# importlib.reload(varseek.utils.varseek_clean_utils)
# from varseek.utils.varseek_clean_utils import make_bus_df

In [18]:
import importlib
import varseek.utils.seq_utils
importlib.reload(varseek.utils.varseek_clean_utils)
from varseek.utils.varseek_clean_utils import make_bus_df
df = make_bus_df(kb_count_out=kb_count_out_test_4_total_fastqs_paired, fastq_file_list=rnaseq_fastq_files_final, t2g_file=test_t2g, mm=False, union=False, technology="bulk", parity="paired", bustools=bustools, check_only=True)

File /Users/joeyrich/Desktop/local/varseek/tests/kb_files/reads1_test.fq does not match the expected bulk file naming convention of SAMPLE_PAIR.EXT where SAMPLE is sample name, PAIR is 1/2, and EXT is a fastq extension - or the Illumina file naming convention of SAMPLE_LANE_R[12]_001.fastq.gz, where SAMPLE is letters, numbers, underscores; LANE is numbers with optional leading 0s; pair is either R1 or R2; and it has .fq or .fastq extension (or .fq.gz or .fastq.gz).
File /Users/joeyrich/Desktop/local/varseek/tests/kb_files/reads2_test.fq does not match the expected bulk file naming convention of SAMPLE_PAIR.EXT where SAMPLE is sample name, PAIR is 1/2, and EXT is a fastq extension - or the Illumina file naming convention of SAMPLE_LANE_R[12]_001.fastq.gz, where SAMPLE is letters, numbers, underscores; LANE is numbers with optional leading 0s; pair is either R1 or R2; and it has .fq or .fastq extension (or .fq.gz or .fastq.gz).
File /Users/joeyrich/Desktop/local/varseek/tests/kb_files/re

Processing FASTQ headers: 3it [00:00, 2437.60it/s]
Processing FASTQ headers: 3it [00:00, 24151.46it/s]
Processing FASTQ headers: 3it [00:00, 7672.51it/s]
Processing FASTQ headers: 3it [00:00, 31223.11it/s]


loading in ec matrix
loading in t2g df
loading in bus df
Merging fastq header df and ec_df into bus df
Apply the mapping function to create gene name columns


100%|██████████| 4/4 [00:00<00:00, 7906.32it/s]


Taking set of gene_names


100%|██████████| 4/4 [00:00<00:00, 8062.09it/s]


Determining what counts in count matrix


100%|██████████| 4/4 [00:00<00:00, 4450.19it/s]

Saving bus df as parquet


In [ ]:
import varseek as vk
vk.ref(
    variants="cosmic",
    sequences="cdna",
    w=11
)

In [11]:
import pandas as pd

# Example DataFrame
df = pd.DataFrame({"vcrs_header": ["tran1:mut1", "tran2:mut2;tran3:mut3"]})

df[["seq_ID", "variant"]] = df["vcrs_header"].apply(lambda x: pd.Series(
    [";".join([s.split(":")[0] for s in x.split(";")]),
     ";".join([s.split(":")[1] for s in x.split(";")])]
))

df

,vcrs_header,seq_ID,variant
0,tran1:mut1,tran1,mut1
1,tran2:mut2;tran3:mut3,tran2;tran3,mut2;mut3


In [16]:
import pandas as pd

df = pd.DataFrame({"variant": ["c.123A>T;c.456G>C", "c.777C>G",  "c.789C>G;c.1011_1012del"]})

# Example mutation pattern (modify as needed)
mutation_pattern = r"(?:c|g)\.([0-9_\-\+\*\(\)\?]+)([a-zA-Z>]+)"  # more complex: r'c\.([0-9_\-\+\*\(\)\?]+)([a-zA-Z>\(\)0-9]+)'

# Function to apply extraction to each semicolon-separated part
def extract_nucleotide_positions_and_actual_variant(variant):
    if pd.isna(variant):
        return pd.Series([None, None])
    
    parts = variant.split(";")  # Split semicolon-separated variants
    extracted = [pd.Series(s).str.extract(mutation_pattern) for s in parts]  # Apply extraction
    extracted_df = pd.concat(extracted, ignore_index=True)  # Combine results
    
    return pd.Series([";".join(extracted_df[col].dropna().astype(str)) for col in extracted_df.columns])

# Apply the function
df[["nucleotide_positions", "actual_variant"]] = df["variant"].apply(extract_nucleotide_positions_and_actual_variant)
df


,variant,nucleotide_positions,actual_variant
0,c.123A>T;c.456G>C,123;456,A>T;G>C
1,c.789C>G,789,C>G
2,c.789C>G;c.1011_1012del,789;1011_1012,C>G;del


In [ ]:
df = pd.DataFrame({"vcrs_header": ["ENST1:c.123A>T;ENST1:c.456G>C", "ENST1:c.777C>G",  "ENST1:c.789C>G;ENST1:c.1011_1012del"]})

import pandas as pd

# Sample DataFrame
df = pd.DataFrame({"vcrs_header": ["ENST1:c.123A>T;ENST1:c.456G>C", "ENST1:c.777C>G", "ENST1:c.789C>G;ENST1:c.1011_1012del"]})

# Step 1: Explode across semicolons
df_exploded = df.assign(vcrs_header=df["vcrs_header"].str.split(";")).explode("vcrs_header")

# Step 2: Apply filtering (example: remove rows containing "del")
df_exploded = df_exploded[~df_exploded["vcrs_header"].str.contains("del", na=False)]
df_exploded = df_exploded.drop(index=1)  # drop the 2nd row

# Step 3: Collapse back, sorting alphabetically
df_collapsed = df_exploded.groupby(df_exploded.index)["vcrs_header"].apply(lambda x: ";".join(sorted(x))).reset_index()

df_collapsed

,index,vcrs_header
0,0,ENST1:c.123A>T;ENST1:c.456G>C
1,2,ENST1:c.789C>G
